## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## HW 5 Pair 9

## Pamela Nguyen and Daniel Carlson

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [626]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
!pip install scikit-learn==1.0

In [627]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [628]:
data = load_diabetes()
df_X = pd.DataFrame(data.data, columns=data.feature_names)
s_y = pd.Series(data.target)
df_X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [629]:
def get_linear_regression_model( df_X, s_y ):
  df_X2 = df_X.copy()
  df_X2.insert(0, 'ones', 1)
  X = df_X2
  # X = df_X
  y = s_y
  beta_hat = np.dot(np.dot( np.linalg.inv( (np.dot(X.T,X) )), X.T),y)
  print(len(df_X.columns))
  print(len(beta_hat))
  return beta_hat

In [630]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

4
5


array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [631]:
def predict_linear_regression_value( beta_hat, df_X ):
  df_X2 = df_X.copy()
  df_X2.insert(0, 'ones', 1)
  X = df_X2
  # print(beta_hat)
  # print(X)
  Y = []
  for index, row in X.iterrows():
    # print(row)
    Y.append(np.matmul(row,beta_hat))
  return Y

In [632]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

[4.036695213679526, 4.3950258942046805, 3.9088470267003004]

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [633]:
def partition_data( df_X, s_y, k ):
  fold_size = len(df_X)/k
  df_X_copy = df_X.copy()
  s_y_copy = s_y.copy()
  dict_k_df_X = {}
  dict_k_s_y = {}
  for i in range(k):
    df_X_fold = df_X_copy[round(i*fold_size):round(i*fold_size+fold_size)]
    s_y_fold = s_y_copy[round(i*fold_size):round(i*fold_size+fold_size)]
    dict_k_df_X[i+1] = df_X_fold.reset_index(drop=True)
    dict_k_s_y[i+1] = s_y_fold.reset_index(drop=True)
  return (dict_k_df_X, dict_k_s_y)

In [634]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [635]:
# Check fold sizes
sum = 0
for i in range(len(dict_k_df_X)):
  sum = sum + len(dict_k_df_X[i+1])
  print('Fold', i+1, 'length of dataframe is', len(dict_k_df_X[i+1]), 'and length of series is', len(dict_k_s_y[i+1]))
print('The sum of the number of elements in each fold is', sum, 'and there are', len(df_X), 'rows in the original df')

Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 89 and length of series is 89
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 89 and length of series is 89
Fold 5 length of dataframe is 88 and length of series is 88
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [636]:
def get_mae( s_y, s_y_hat):
  mae = 0
  n = len(s_y)
  for i in range(1,len(s_y)+1):
    mae = mae + ((abs(s_y[i-1]-s_y_hat[i-1]))/n)
  return mae

In [637]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [638]:
mae = np.array([])
for k in dict_k_df_X.keys():
  beta_hat = get_linear_regression_model(dict_k_df_X[k], dict_k_s_y[k])
  s_y_hat = predict_linear_regression_value(beta_hat, dict_k_df_X[k])
  s_y = dict_k_s_y[k]
  mae = np.append( mae, get_mae(s_y,s_y_hat) )  

10
11
10
11
10
11
10
11
10
11


In [639]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 38.92, the max MAE is 44.35, and the mean MAE is 41.57


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [640]:
iris = load_iris(as_frame=True)
df_X = iris.data
s_y = iris.target
print(df_X)
print(s_y)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]
0      0
1      0
2      0
3      0
4   

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [641]:
k = 5
dict_k_df_X,dict_k_s_y  = partition_data( df_X, s_y, k )

In [642]:
# Check fold sizes
sum = 0
for i in range(len(dict_k_df_X)):
  sum = sum + len(dict_k_df_X[i+1])
  print('Fold', i+1, 'length of dataframe is', len(dict_k_df_X[i+1]), 'and length of series is', len(dict_k_s_y[i+1]))
print('The sum of the number of elements in each fold is', sum, 'and there are', len(df_X), 'rows in the original df')

Fold 1 length of dataframe is 30 and length of series is 30
Fold 2 length of dataframe is 30 and length of series is 30
Fold 3 length of dataframe is 30 and length of series is 30
Fold 4 length of dataframe is 30 and length of series is 30
Fold 5 length of dataframe is 30 and length of series is 30
The sum of the number of elements in each fold is 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [643]:
def get_acc( s_1, s_2 ):
    return (len(s_1[s_1 == s_2]))/len(s_1)

In [644]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [645]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    print(F'k = {k}')
    test_X = dict_k_df_X[k]
    inner_foldX = dict_k_df_X.copy()

    test_Y = dict_k_s_y[k]
    inner_foldY = dict_k_s_y.copy()

    fold_acc, fold_avg = np.array([]), np.array([])
    for pos_min_impurity in possible_min_impurity_decrease:
        print(F'Testing {pos_min_impurity} min impurity decrease')

        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        fold_acc = np.array([])
        for fold in inner_foldX.keys():
          train_X = pd.concat([val for key, val in inner_foldX.items() if key != fold]).reset_index(drop=True)
          train_Y = pd.concat([val for key, val in inner_foldY.items() if key != fold]).reset_index(drop=True)
          
          clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=pos_min_impurity)
          clf.fit(train_X,train_Y)
          #acc = get_acc(clf.predict(test_X), test_Y)
          acc = get_acc(clf.predict(train_X),train_Y)
          #acc = (clf.predict(train_X[fold])==train_Y[fold]).sum()/len(train_Y[fold])
          fold_acc = np.append(fold_acc, acc)
        
        print('Average accuracy over 4 folds is: {:.2f}'.format( np.average(fold_acc)))
        fold_avg = np.append(fold_avg, np.average(fold_acc))
        #print(np.max(fold_acc))
        #print(np.min(fold_acc))
        max_imp_ind = np.argmax(fold_avg)
        #print(fold_avg)
        max_imp = possible_min_impurity_decrease[max_imp_ind]

    #print(np.max(fold_avg))
    print()
    print(F'Best min impurity decrease is {max_imp}')
    print()



    # Use best min impurity decrease to train model
    
    # outer accuracy calculation 
    clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=max_imp)
    clf.fit(train_X,train_Y)
    this_acc = get_acc(clf.predict(test_X),test_Y)
    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop
    #print(this_acc)

min, max, avg = np.min(outer_acc), np.max(outer_acc), np.average(outer_acc)

k = 1
Testing 0.1 min impurity decrease
Average accuracy over 4 folds is: 0.97
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is: 0.83
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is: 0.72
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is: 0.42

Best min impurity decrease is 0.1

k = 2
Testing 0.1 min impurity decrease
Average accuracy over 4 folds is: 0.97
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is: 0.83
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is: 0.72
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is: 0.42

Best min impurity decrease is 0.1

k = 3
Testing 0.1 min impurity decrease
Average accuracy over 4 folds is: 0.97
Testing 0.25 min impurity decrease
Average accuracy over 4 folds is: 0.83
Testing 0.3 min impurity decrease
Average accuracy over 4 folds is: 0.72
Testing 0.4 min impurity decrease
Average accuracy over 4 folds is: 0.42

Best min impurity decrease is 

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [646]:
print('Accuracy of test sets: Min = {:.2f}, Max = {:.2f}, Mean = {:.2f}'.format(min, max, avg))

Accuracy of test sets: Min = 0.83, Max = 1.00, Mean = 0.95
